In [1]:
################################################################################
# MIT License
#
# Copyright (c) 2021 University of Amsterdam
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to conditions.
#
# Author: Deep Learning Course (UvA) | Fall 2022 & Oliver Gurney-Champion | Spring 2023
# Date modified: Jan 2023
################################################################################


# Import requiered packages
imports the packages and sets the random seed

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import random
import numpy as np
import helper_functions as hf
import torch
from matplotlib import pyplot as plt

# set random seed
seed =42

login to your free wandb account. Note you will need to set up your account on https://wandb.ai/authorize
wandb allows you to keep track of your neural network training.

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: oliverchampion (dl4mi). Use `wandb login --relogin` to force relogin


True

# Simulate and view the IVIM data
Here, we split our data into a training set, validation set and test set. Note that the current implementation only uses the training set and it is up to you (in your exercises) to also implement the validation and test run. At this point, we already split the data for you.

In [ ]:
# set b-values at which we "measure" (i.e. simulate signal)
bvalues=[0, 10, 20, 30, 50, 75, 100, 150, 300, 500, 700, 1000]

## Set the random seeds for reproducibility
np.random.seed(seed)
random.seed(seed)

#determine
batch_size = 128

with wandb.init(project="AI_for_medical_imaging", job_type="visualize data") as run:
    data_sim = hf.sim_signal(SNR=(10,40),bvalues=bvalues,sims=10000,seed=np.random.randint(1,10000))
    # Only for visualisation purposes: here we create our "Artifact" in wandb --> this allows viewing the data in your wandb account
    for i in range(4):
        #make b-value data pairs
        example_data=[[x,y] for (x,y) in zip(bvalues,data_sim[0][i])]
        # put it in a table
        table = wandb.Table(data=example_data, columns=["b-values", "signal"])
        #tell wandb to plot the table
        wandb.log({"data_plot " + str(i): wandb.plot.scatter(table, "b-values", "signal")})

    # here we split the data into train (70%), test (15%) and validation (15%) sets
    #split = int(np.floor(len(data_sim[0]) * 0.7))
    train_set, test_set, val_set = torch.utils.data.random_split([[data_sim[0][i,:],data_sim[1][i],data_sim[2][i],data_sim[3][i]] for i in range(len(data_sim[3]))],[0.7,0.15,0.15])
    #split = int(np.floor(len(rest) * 0.5))
    #test_set, val_set = torch.utils.data.random_split([[rest[0][i,:],rest[1][i],rest[2][i],rest[3][i]] for i in range(len(rest[3]))],[split, len(rest[0]) - split])

    # train loader loads the trianing data. We want to shuffle to make sure data order is modified each epoch and different data is selected each epoch.
    trainloader = torch.utils.data.DataLoader(train_set,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=True)
    # validation data is loaded here. By not shuffling, we make sure the same data is loaded for validation every time. We can use substantially more data per batch as we are not training.
    inferloader = torch.utils.data.DataLoader(val_set,
                                   batch_size=batch_size,
                                   shuffle=False,
                                   drop_last=True)
        # validation data is loaded here. By not shuffling, we make sure the same data is loaded for validation every time. We can use substantially more data per batch as we are not training.
    testloader = torch.utils.data.DataLoader(test_set,
                                   batch_size=batch_size,
                                   shuffle=False,
                                   drop_last=True)

# 3B: Design a neural network
Combine all modules into a multilayer perceptron (MLP) in “exercise_3.ipynb”. Use ReLU activation functions after each fully connected linear module, except for the last module, after which a Tanh module should be used (constraining parameters between [-1, 1]). The number of layers and number of nodes per layer should be adjustable.

In [6]:
"""
This module implements a multi-layer perceptron (MLP) in NumPy.
You should fill in code into indicated sections.
"""
from modules import MLP #you need to adapt these





# Exercise 3C-D
C.	Train the neural network using your code in “exercise_3.ipynb”. As a sanity check, a network with 2 layers (64, 32), combined with a lr of 0.001, 30 epochs, and a batch size of 128 should work reasonably well and result in a systematic error of around 0.2% and a random error of around 20%.
D.	It now uses the mini-batch stochastic gradient descent algorithm. Add momentum to the training update of the weights.


In [ ]:
# define parameters
hidden_layers=(64,32)
model=MLP(len(bvalues), hidden_layers, 1)
epochs = 30
learning_rate = 0.001

# initialize wandb
wandb.init(
        project="AI_for_medical_imaging", job_type="training")

loss_module1 = MSE()

# set random seed for reproducibility
torch.manual_seed(seed)

# probe available devices
if torch.cuda.is_available():  # GPU operation have separate seed
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False

# loop over epochs
for epoch in range(epochs):
    # initiate losses to 0
    train_loss_f = 0
    val_loss_f = 0
    #loop over all training data
    for x in trainloader:
        # get data (x[0]) and put the data on the GPU if available
        batch=x[0].numpy()
        # get the reference f (x[2]) --> note x[1] and x[3] are D and Dp respectively
        f_ref = np.expand_dims(x[2].numpy(),axis=1)
        # put the data through the neural network
        f_pred = model.forward(batch)
        # calculate loss (compare predicted f to the ground trueth)
        train_loss_f += np.mean(loss_module1.forward(f_pred, f_ref))
        # propogate the loss through the network (calculate d_wights/d_loss)
        model.backward(loss_module1.backward())

        # update all weights accoording to their derrivatives to the loss.
        for objs in model.LM:
            objs.bias = objs.bias - np.transpose([learning_rate * objs.grads['bias']])
            objs.weights = objs.weights - learning_rate * objs.grads['weight']
        model.clear_cache()
    # initialize error_metrics
    SD_val=0
    sys_val=0
    wandb.log({"loss/train": train_loss_f/trainloader.__len__(),"loss/val": val_loss_f/inferloader.__len__(),"error/random error":SD_val,"error/systematic error":sys_val})
    print('epoch = ' + str(epoch) + ' train loss =' + str(train_loss_f/trainloader.__len__()) +' val loss =' + str(val_loss_f/inferloader.__len__()) + 'the systematic error is ' + str(sys_val) + ' and the random error is ' + str(SD_val))